## Houston zip code
Rice Military 77007
Katy 77494
Methods
- Divide houston metropolitan area into 2 counties: Harris county and Fort Bend county

In [5]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import time
import webdriver
from random import seed, randint, random

ModuleNotFoundError: No module named 'webdriver'

In [10]:
def dstr_2_int(dstr):
    return int(dstr[1:].replace(',',''))
def get_p_z_zr_mc(soup):
    soup_p = soup.select('div.ds-chip>div.ds-home-details-chip>div.ds-summary-row-container>div.ds-summary-row-content>div.ds-summary-row>h4>span>span')
    if soup_p:
        if len(soup_p) > 1: # "From 399,000" From is in the first <span>
            Price = dstr_2_int(soup_p[-1].text)
        else:
            Price = dstr_2_int(soup_p[0].text)
    else:
        Price = 0

    soup_zm = soup.select('div#ds-home-values div.ds-expandable-card-section-default-padding>div>div>div>div>span')
    if soup_zm:
        Zestmate = dstr_2_int(soup_zm[0].text)
    else:
        Zestmate = 0
    
    soup_zmr = soup.select('div#ds-rental-home-values span.Text-c11n-8-18-0__aiai24-0.bloUvX')
    if soup_zmr:
        Zestmate_Rent = dstr_2_int(soup_zmr[0].text.replace('/mo', ''))
    else:
        Zestmate_Rent = 0
   
    soup_mc = soup.select('li.ds-data-view-item>div>div.ds-expandable-card-section-default-padding>div.ds-expandable-card>div>h5')
    #('h5.Text-c11n-8-18-0__aiai24-0.StyledHeading-c11n-8-18-0__ktujwe-0.LJxVR.sc-qQmou.bViFbQ')
    if soup_mc:
        Monthly_Cost = dstr_2_int(soup_mc[0].text)
    else:
        Monthly_Cost = 0
    return Price, Zestmate, Zestmate_Rent, Monthly_Cost
def get_school_info(soup):
    soup_sh = soup.select('span#skip-link-schools+div li>div:first-of-type>div>span:first-of-type')
    if soup_sh:
        scores = map(lambda x: x.text, soup_sh)
        scores_sum = 0
        try:
            scores_int = map(lambda x: int(x.text), soup_sh)
            scores_sum = sum(scores_int)
        except:
            pass
        return ','.join(scores), scores_sum
    else:
        return '', 0

def extract_zillow_info(df_out, path_out, n_start=0, driver_type='selenium', sec_sleep=3):
    cols = ['zestmate', 'zestmate_rent', 'monthly_cost', 'school', 'school_sum', 'monthly_cash_flow', 'yearly_return']
    for c in cols:
        df_out[c] = np.nan
    for i, row in df_out[n_start:].iterrows():
        url = row['url']
        # print(url)
        soup = webdriver.get_soup(url, driver_type=driver_type)
        Price, Zestmate, Zestmate_Rent, Monthly_Cost = get_p_z_zr_mc(soup)
        sch_scores, sch_scores_sum = get_school_info(soup)
        # print(Price, Zestmate, Zestmate_Rent, Monthly_Cost)
        # print(sch_scores)
        df_out.loc[i, ['zestmate', 'zestmate_rent', 'monthly_cost', 'school', 'school_sum']] = Zestmate, Zestmate_Rent, Monthly_Cost, sch_scores, sch_scores_sum
        if Zestmate_Rent != 0 and Monthly_Cost != 0:
            df_out.loc[i, 'monthly_cash_flow'] = Zestmate_Rent - Monthly_Cost
            # the yearly return assumes 20% down payment and 2% overall closing fee
            df_out.loc[i, 'yearly_return'] = round((Zestmate_Rent - Monthly_Cost) * 12 / (Price * 0.22), 3)
        if i % 50 == 0:
            df_out.to_csv(path_out)
            print(f'{i} homes have been scaped ....')
        time.sleep(random() * sec_sleep  
        df_out.to_csv(path_out)

### Get all Fort Bend info

In [1]:
# df = pd.read_csv(r'data/dataset_FortBend_2021-01-10.csv')
# cols = [c for c in df.columns if 'photo' not in c]
# df_out = df[cols]
from import_data import load_json_homes
folder = r'/Users/yuchenzhang/Documents/Reps/actor-zillow-api-scraper/apify_storage/datasets/default'
df_out = load_json_homes(folder)

In [6]:

df_out['zestmate'], df_out['zestmate_rent'], df_out['monthly_cost'] = [np.nan, np.nan, np.nan]
df_out['monthly_cash_flow'] = np.nan
df_out['yearly_return'] = np.nan
df_out['school'] = np.nan

In [7]:
# filter out land with no bedroom or bathroom
df_out = df_out[(df_out.bedrooms) > 0 & (df_out.bathrooms > 0)].reset_index()